In [3]:
#Step-1: Importing the dataset
import pandas as pd
import numpy as np

In [4]:
#Step-2: Reading the dataset
train_data = pd.read_csv("D:\\Machine Learning\\Datasets\\Decision Tree Data\\PlayTennis.csv")
train_data.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [5]:
#Step-3: Calculating the entropy of the whole dataset

def calculate_total_entropy(train_data, label, class_list):
    no_of_rows = train_data.shape[0] #the total size of the dataset
    total_entropy = 0
    
    for c in class_list : #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0]
        #number of the class
        total_class_entropy = -(total_class_count/no_of_rows)*np.log2(total_class_count/no_of_rows)
        #entropy of the class
        total_entropy += total_class_entropy
        #adding the class entropy to the total entropy of the dataset
        
    return total_entropy

In [6]:
#Step-4: Calculating the entropy of the filtered dataset

def calculate_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list : 
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0]
        #row count of the class c
        entropy_class = 0
        if label_class_count != 0:
            prob_class = label_class_count/class_count
            #probability of the class
            entropy_class = -(prob_class)*np.log2(prob_class)
            #entropy 
        entropy += entropy_class
        
    return entropy

In [7]:
#Step-5: Calculating information gain for a feature

def calculate_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() 
    #unqiue values of the feature
    no_of_rows = train_data.shape[0]
    feature_info_gain = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] 
        #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calculate_entropy(feature_value_data, label, class_list) 
        #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/no_of_rows
        feature_info_gain += feature_value_probability * feature_value_entropy 
        #calculating information of the feature value
        
    return calculate_total_entropy(train_data, label, class_list) - feature_info_gain 
    #calculating information gain by subtracting


In [8]:
#Step-6: Finding the most informative feature (feature with highest information gain)

def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) 
    # finding the feature names in the dataset
    # label is not a feature, so dropping it
    
    max_info_gain = -1
    max_info_feature = None
    
    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calculate_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: 
            #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
            
    return max_info_feature

In [9]:
#Step-7: Adding a node to the tree

def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) 
    #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] 
        #dataset with only feature_name = feature_value
        
        assigned_to_node = False 
        #flag for tracking feature_value is pure class or not
        
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] 
            #count of class c
            if class_count == count: 
            # count of feature_value = count of class (pure class)
                tree[feature_value] = c 
                #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] 
                #removing rows with feature_value
                assigned_to_node = True
                
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" 
            #not a pure classification -- should extend the node, so the branch is marked with ?
            
    return tree, train_data

In [10]:
#Step-8: To create the final tree

def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: 
    #if dataset becomes empty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) 
        #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) 
        #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: 
        #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        
        else: 
        #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): 
        #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] 
                #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) 
                #recursive call with updated dataset

In [11]:
#Step-9: Finding unique classes of the label and Starting the algorithm

def id3(train_data_m, label):
    train_data_copy = train_data.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data_copy[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

In [12]:
#Step-10:Printing the tree
print("INFERENCE RULE")
tree = id3(train_data, 'Play Tennis')
print(tree)

INFERENCE RULE
{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


In [13]:
#Step-11: Prediction

def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

In [14]:
instance1 = train_data.iloc[2]   #3rd instance of the dataset
prediction = predict(tree,instance1)
print(prediction)

#Here, the third instance of the dataset is ['Overcast','Hot','High','Weak'] for which the label should be 'YES'

Yes


In [15]:
#Step-12: Accuracy of the model

def evaluate(tree, test_data, label):
    correct_predict = 0
    wrong_predict = 0
    for index, row in test_data.iterrows(): #for each row in the dataset
        result = predict(tree, test_data.iloc[index]) #predict the row
        if result == test_data[label].iloc[index]: 
        #predicted value and expected value is same or not
            correct_predict += 1 #increase correct count
        
        else:
            wrong_preditct += 1 #increase incorrect count
            
    #calculating accuracy
    accuracy = correct_predict / (correct_predict + wrong_predict) 
    return accuracy

In [16]:
test_data = pd.read_csv("D:\\Machine Learning\\Datasets\\Decision Tree Data\\PlayTennis_test data.csv")
test_data.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Overcast,Mild,Normal,Weak,Yes
1,Sunny,Mild,Normal,Weak,Yes
2,Rain,Mild,Normal,Weak,Yes
3,Rain,Cool,High,Strong,No


In [19]:
for i in range(test_data.shape[0]):
    instance1 = test_data.iloc[i]   #3rd instance of the dataset
    prediction = predict(tree,instance1)
    print("\n\nInstance - 1 :")
    print(prediction)



Instance - 1 :
Yes


Instance - 1 :
Yes


Instance - 1 :
Yes


Instance - 1 :
No


In [20]:
#evaluating the accuracy of the test dataset

accuracy = evaluate(tree, test_data, 'Play Tennis')
print(accuracy)

1.0
